In [3]:
import requests
import pandas as pd
import numpy as np

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import time
import os
from datetime import datetime

In [5]:
provinces = ['BKK','Chiangmai','Khonkaen','Rayong','Saraburi','Surat']
provincesLatLng = dict()

In [6]:
for province in provinces:
    file = open(f'./{province}/position.txt','r').read().split('\n')
    temp = dict()
    temp['lat'] = file[0].split(' ')[2]
    temp['lng'] = file[1].split(' ')[2]
    provincesLatLng[province] =  temp
provincesLatLng

{'BKK': {'lat': '13.729984', 'lng': '100.536443'},
 'Chiangmai': {'lat': '18.840633', 'lng': '98.969661'},
 'Khonkaen': {'lat': '16.445329', 'lng': '102.835251'},
 'Rayong': {'lat': '12.671521', 'lng': '101.275875'},
 'Saraburi': {'lat': '14.685833', 'lng': '100.871996'},
 'Surat': {'lat': '9.126057', 'lng': '99.325355'}}

In [7]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been s

In [8]:
# 2020/07/01 01:00(utc+7) – 2021/06/30 22:00 => 2020/06/30/1800Z (utc+0) - 2021/06/30/1500Z (utc+0)
def formatTime(dt):
    temp = dt.split(' ')
    date0, time0 = temp[0] , temp[1]
    year,month,day = date0.split('-')
    date = ("/").join([day,month,year])
    hour = time0.split(':')[0]
    time = (":").join([str(int(hour)),'00','00'])
    date_time = (" ").join([date,time])
    return date_time

In [9]:
formatTime("2020-06-30 07:00 Local")

'30/06/2020 7:00:00'

In [10]:
#use time in utc+0
def scrapeTemp(year,month,day,hour,province):
    lat, lng = provincesLatLng[province]['lat'], provincesLatLng[province]['lng']
    url = f'https://earth.nullschool.net/#{year}/{month:02d}/{day:02d}/{hour:02d}00Z/wind/surface/level/anim=off/overlay=temp/orthographic/loc={lng},{lat}'
    driver.get(url=url)
    element = WebDriverWait(driver,9999).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="spotlight-panel"]/div[3]/div')))
    data_status = driver.find_element(By.XPATH,'/html/body/main/div[3]/div[1]/div')
    if data_status.text=="Downloading...":
        while True:
            time.sleep(0.05)
            data_status = driver.find_element(By.XPATH,'/html/body/main/div[3]/div[1]/div')
            if data_status.text=="Downloading...":
                continue
            else :
                break
    temperature = element.text.split(' ')[0]
    print(temperature)
    return temperature

In [11]:
scrapeTemp(2020,6,30,6,'BKK')

32.0


'32.0'

In [ ]:
nday = [0,31,28,31,30,31,30,31,31,30,31,30,31]

In [ ]:
# 2020/07/01 01:00(utc+7) – 2021/06/30 22:00 => 2020/06/30/1800Z (utc+0) - 2021/06/30/1500Z (utc+0)
def getTempData(province):
    tempDict = dict()
    listTemp = []
    y = 2020
    temperature = scrapeTemp(2020,6,30,18,province)
    listTemp.append(temperature)
    temperature  = scrapeTemp(2020,6,30,21,province)
    listTemp.append(temperature)
    for m in range(7,13):
        for d in range(1,32):
            if(d > nday[m]):
                continue
            for h in range(0,24,3):
                temperature= scrapeTemp(y,m,d,h,province)
                listTemp.append(temperature)
    y = 2021
    for m in range(1,7):
        for d in range(1,32):
            if(d > nday[m]):
                continue
            for h in range(0,24,3):
                temperature= scrapeTemp(y,m,d,h,province)
                listTemp.append(temperature)
                if(m==6 and d== 30 and h == 15):
                    break
    tempDict['temp'] = listTemp
    return tempDict

In [ ]:
tempDict = dict()
listDateTime = []
listTemp = []
temperature = scrapeTemp(2020,6,30,18,'BKK')
listTemp.append(temperature)
temperature = scrapeTemp(2020,6,30,6,'BKK')
listTemp.append(temperature)
tempDict['temp'] = listTemp
tempDict

In [ ]:
# 2021/07/01 01:00(utc+7) – 2021/07/01 22:00 => 2021/06/30/1800Z (utc+0) - 2021/07/01/1500Z (utc+0)
def scrapeAdditionalDay(province):
    df1 = pd.read_csv(f'./{province}_temp_surface_test.csv')
    tempDict2 = dict()
    listTemp2 = []
    # 01
    temperature= scrapeTemp(2021,6,30,18,province)
    listTemp2.append(temperature)
    # 04
    temperature= scrapeTemp(2021,6,30,21,province)
    listTemp2.append(temperature)
    # 07
    temperature= scrapeTemp(2021,7,1,0,province)
    listTemp2.append(temperature)
    # 10
    temperature= scrapeTemp(2021,7,1,3,province)
    listTemp2.append(temperature)
    # 13
    temperature= scrapeTemp(2021,7,1,6,province)
    listTemp2.append(temperature)
    # 16
    temperature= scrapeTemp(2021,7,1,9,province)
    listTemp2.append(temperature)
    # 19
    temperature= scrapeTemp(2021,7,1,12,province)
    listTemp2.append(temperature)
    # 22
    temperature= scrapeTemp(2021,7,1,15,province)
    listTemp2.append(temperature)
    tempDict2['temp'] = listTemp2
    df2 = pd.DataFrame(tempDict2)
    return pd.concat([df1,df2])

In [ ]:
tempBKK = getTempData('BKK')

In [ ]:
dfBKK = pd.DataFrame(tempBKK)
dfBKK.to_csv('bkk_temp_surface_test.csv', index=False)

In [ ]:
dfBKKNew = scrapeAdditionalDay('BKK')
dfBKKNew.to_csv('bkk_temp_surface_test2.csv', index=False)

In [ ]:
tempChiangmai = getTempData('Chiangmai')

In [ ]:
dfChiangmai = pd.DataFrame(tempChiangmai)
dfChiangmai.to_csv('chiangmai_temp_surface_test.csv', index=False)

In [ ]:
dfChiangmaiNew = scrapeAdditionalDay('Chiangmai')
dfChiangmaiNew.to_csv('Chiangmai_temp_surface_test2.csv', index=False)

In [ ]:
tempKhonkaen = getTempData('Khonkaen')

In [ ]:
dfKhonkaen = pd.DataFrame(tempKhonkaen)
dfKhonkaen.to_csv('khonkaen_temp_surface_test.csv', index=False)

In [ ]:
dfKhonkaenNew = scrapeAdditionalDay('Khonkaen')
dfKhonkaenNew.to_csv('khonkaen_temp_surface_test2.csv', index=False)

In [ ]:
tempRayong = getTempData('Rayong')

In [ ]:
dfRayong = pd.DataFrame(tempRayong)
dfRayong.to_csv('rayong_temp_surface_test.csv', index=False)

In [ ]:
dfRayongNew = scrapeAdditionalDay('Rayong')
dfRayongNew.to_csv('rayong_temp_surface_test2.csv', index=False)

In [ ]:
tempSaraburi = getTempData('Saraburi')

In [ ]:
dfSaraburi = pd.DataFrame(tempSaraburi)
dfSaraburi.to_csv('saraburi_temp_surface_test.csv', index=False)

In [ ]:
dfSaraburiNew = scrapeAdditionalDay('Saraburi')
dfSaraburiNew.to_csv('Saraburi_temp_surface_test2.csv', index=False)

In [ ]:
tempSurat = getTempData('Surat')

In [ ]:
dfSurat = pd.DataFrame(tempSurat)
dfSurat.to_csv('surat_temp_surface_test.csv', index=False)

In [ ]:
dfSuratNew = scrapeAdditionalDay('Surat')
dfSuratNew.to_csv('Surat_temp_surface_test2.csv', index=False)

In [ ]:
driver.quit()